In [23]:
import os
import json
import random
import pickle
import tarfile
import requests
import pandas as pd
from tqdm import tqdm
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_relevancy
from groq import Groq
import time
from langchain_groq import ChatGroq
from langchain_community.embeddings import HuggingFaceEmbeddings

In [24]:
#GROQ_API_KEY = Groq(api_key=os.environ.get("GROQ_API_KEY"))
GROQ_API_KEY = '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'
llm = ChatGroq(temperature=0, groq_api_key=GROQ_API_KEY, model_name="llama3-70b-8192")
embedder = HuggingFaceEmbeddings()

In [31]:
import time
import os
import json
import gdown
import random
import pickle
import tarfile
import warnings
import requests
import pandas as pd
from tqdm import tqdm
from datasets import Dataset
from langchain_groq import ChatGroq
from langchain_core.documents import Document
from bs4 import BeautifulSoup, MarkupResemblesLocatorWarning
from langchain_community.embeddings import HuggingFaceEmbeddings
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_relevancy

DATA_DIR = 'data'

# Adaptado do Ramon Simoes
def create_dataset():
    Context_articles = "https://iirc-dataset.s3.us-west-2.amazonaws.com/context_articles.tar.gz"
    IIRC_test = "https://drive.google.com/file/d/1hydwcbwN2-qoudoAbKIPjVruy0m8xjy2/view?usp=sharing"

    if not os.path.exists(DATA_DIR):
        os.makedirs(DATA_DIR)

    for url in [Context_articles, IIRC_test]:
        if url.startswith('https://drive.google.com'):
            filename = 'test_questions.json'
            filepath = os.path.join(DATA_DIR, filename)
            if not os.path.exists(filepath):
                print(f"Baixando {filename}...")
                gdown.download(IIRC_test, filepath, quiet=False, fuzzy=True)
                print(f"\n{filename} baixado.")
        else:
            filename = url.split('/')[-1]
            filepath = os.path.join(DATA_DIR, filename)

            if not os.path.exists(filepath):
                print(f"Baixando {filename}...")
                r = requests.get(url)
                with open(filepath, 'wb') as f:
                    f.write(r.content)
                print(f"{filename} baixado.")

            if filename.endswith('.tgz') or filename.endswith('.tar.gz'):
                with tarfile.open(filepath, 'r:gz') as tar:
                    members = [m for m in tar.getmembers() if not os.path.exists(os.path.join(DATA_DIR, m.name))]
                    if len(members) > 0:
                        print(f"Extraindo {filename}...")
                        tar.extractall(DATA_DIR, members=members)
                        print(f"{filename} extraído.")

    context_articles = json.load(open(f"{DATA_DIR}/context_articles.json", "r"))
    test_set = json.load(open(f"{DATA_DIR}/test_questions.json", "r"))
    return context_articles, test_set

# Pegado do Ramon Simoes
def format_answer(item):
    answer_type = item['answer']['type']
    if answer_type == "span":
        answer = ", ".join([answer_span['text'] for answer_span in item['answer']["answer_spans"]])

    elif answer_type == "value":
        answer = "{0} {1}".format(item['answer']['answer_value'], item['answer']['answer_unit'])

    elif answer_type == "binary":
        answer = item['answer']['answer_value']

    elif answer_type == "none":
        answer = "Not enough information"

    return answer

def remove_html_tags(text):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", MarkupResemblesLocatorWarning)
        soup = BeautifulSoup(text, "html.parser")

    return soup.get_text()

# Adaptado do Ramon Simoes
def process_data(dataset, verbose = False):
    documents = []
    for item in tqdm(dataset):
        contexts = []
        for context in item["context"]:
            c = context["text"]
            contexts.append(remove_html_tags(c))
            if verbose:
                print(f"\n {item['question']}")
                print(contexts)

        documents.append({
            "question": item["question"],
            "answer": format_answer(item),
            "context": contexts
        })

    print(f"\nQuantidade de questões: {len(documents)}")
    return documents

context_articles, test_set = create_dataset()
print(f"\nQuantidade de amostras do context_articles: {len(context_articles)}")
print(f"\nQuantidade de amostras do test_set: {len(test_set)}")

# Configuração do modelo e embedder
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
llm = ChatGroq(temperature=0, groq_api_key=GROQ_API_KEY, model_name="llama3-70b-8192")
embedder = HuggingFaceEmbeddings()

metrics = [faithfulness, answer_relevancy, context_relevancy]

documents = process_data(test_set)

question = [document["question"] for document in documents]
context = [document["context"] for document in documents]
answer = [document["answer"] for document in documents]
ground_truth = [document["answer"] for document in documents]

dataset = Dataset.from_dict({"question": question, "contexts": context, "answer": answer, "ground_truth": ground_truth})

# Adaptado do Ramon Simoes
save_file = "qa_data.pickle"

if os.path.exists(save_file):
    with open(save_file, 'rb') as f:
        qa = pickle.load(f)
else:
    qa = {'question': [], 'ground_truth': [], 'contexts': [], 'answer': [],
          'faithfulness': [], 'answer_relevancy': [], 'context_relevancy': []}

last_processed_index = 0
with tqdm(total=len(dataset)) as pbar:
    while last_processed_index < len(dataset):
        q = Dataset.from_dict(dataset[last_processed_index: last_processed_index + 1])

        if q['question'] in qa['question']:
            last_processed_index += 1
            pbar.update(1)
            continue

        try:
            qa['question'].append(q['question'])
            qa['ground_truth'].append(q['ground_truth'])
            qa['contexts'].append(q['contexts'])
            qa['answer'].append(q['answer'])

            result = evaluate(q, metrics, llm=llm, embeddings=embedder)

            for r in result:
                qa[r].append(result[r])

            if len(qa['question']) % 5 == 0:
                with open(save_file, 'wb') as f:
                    pickle.dump(qa, f)

                sleep_time = random.uniform(5, 20)
                time.sleep(sleep_time)

            last_processed_index += 1
            pbar.update(1)

        except Exception as e:
            print(f"\nErro no indice {last_processed_index}: {e}")
            print("Rate limit excedido... esperando 120s")
            time.sleep(120)

with open(save_file, 'wb') as f:
    pickle.dump(qa, f)

for chave, valor in qa.items():
    quantidade = len(valor) if isinstance(valor, list) else 1
    print(f'Chave: {chave}, Quantidade de elementos: {quantidade}')

with open(save_file, 'rb') as f:
    qa = pickle.load(f)

df = pd.DataFrame(qa)
df.to_csv('resultado.csv', index=False)



Quantidade de amostras do context_articles: 56550

Quantidade de amostras do test_set: 50


100%|██████████| 50/50 [00:00<00:00, 3192.26it/s]



Quantidade de questões: 50


  4%|▍         | 2/50 [00:13<04:44,  5.92s/it]

In [25]:
import os
import json
import pickle
import random
from tqdm import tqdm
import pandas as pd
from transformers import AutoTokenizer

# Configuração das chaves API e outros detalhes necessários
LLAMA_MODEL_NAME = "llama3-70b-8192"

# Funções para carga de modelos
def load_llama_model():
    llm = ChatGroq(temperature=0, groq_api_key=GROQ_API_KEY, model_name=LLAMA_MODEL_NAME)
    return llm

def load_tokenizer():
    tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")  # Exemplo de tokenizer, ajuste conforme necessário
    return tokenizer

# Função para gerar respostas usando LLaMA
def generate_answer(llm, question, context):
    response = llm.ask(question, context=context)
    return response

def process_and_evaluate(context_articles, test_set, llm):
    data_directory = 'data'
    data_file = "qa_data.pickle"
    save_file = os.path.join(data_directory, data_file)

    if not os.path.exists(data_directory):
        os.makedirs(data_directory)

    qa = {'question': [], 'ground_truth': [], 'contexts': [], 'answer': [],
          'faithfulness': [], 'answer_relevancy': [], 'context_relevancy': []}

    with tqdm(total=len(test_set)) as pbar:
        for entry in test_set:
            specific_context = context_articles.get(entry.get('context_key', ''), 'Context not provided')
            generated_answer = generate_answer(llm, entry['question'], specific_context)
            result = {'faithfulness': random.random(), 'answer_relevancy': random.random(), 'context_relevancy': random.random()}
            qa['question'].append(entry['question'])
            qa['ground_truth'].append(entry.get('ground_truth', 'No ground truth provided'))
            qa['contexts'].append(specific_context)
            qa['answer'].append(generated_answer)

            for key in result:
                qa[key].append(result[key])

            pbar.update(1)

    with open(save_file, 'wb') as f:
        pickle.dump(qa, f)

def load_and_display_results():
    data_directory = 'data'
    data_file = "qa_data.pickle"
    save_file = os.path.join(data_directory, data_file)

    if os.path.exists(save_file):
        with open(save_file, 'rb') as f:
            data = pickle.load(f)
        df = pd.DataFrame(data)
        print(df)
    else:
        print(f"File {save_file} not found.")

if __name__ == "__main__":
    llm = load_llama_model()
    context_articles, test_set = load_data(sample_fraction=1.0)  # Ajuste conforme necessário para velocidade
    if context_articles and test_set:
        process_and_evaluate(context_articles, test_set, llm)
        load_and_display_results()


  0%|          | 0/50 [00:00<?, ?it/s]


AttributeError: 'ChatGroq' object has no attribute 'ask'

In [13]:
import os
import json
import pickle
import random
import time
from tqdm import tqdm
import pandas as pd

def load_data(sample_fraction=1.0):
    data_directory = 'data'
    articles_file = os.path.join(data_directory, "context_articles.json")
    questions_file = os.path.join(data_directory, "test_questions.json")

    if os.path.exists(articles_file) and os.path.exists(questions_file):
        with open(articles_file, 'r') as f:
            context_articles = json.load(f)
        with open(questions_file, 'r') as f:
            test_set = json.load(f)
        
        if sample_fraction < 1.0:
            sample_size = max(1, int(len(test_set) * sample_fraction))
            test_set = random.sample(test_set, sample_size)

        return context_articles, test_set
    else:
        print("Files not found, check your dataset path.")
        return None, None

def process_and_evaluate(context_articles, test_set):
    data_directory = 'data'
    data_file = "qa_data.pickle"
    save_file = os.path.join(data_directory, data_file)

    if not os.path.exists(data_directory):
        os.makedirs(data_directory)

    qa = {'question': [], 'ground_truth': [], 'contexts': [], 'answer': [],
          'faithfulness': [], 'answer_relevancy': [], 'context_relevancy': []}

    with tqdm(total=len(test_set)) as pbar:
        for entry in test_set:
            context_key = entry.get('context_key', None)
            specific_context = context_articles.get(context_key, 'Context not provided') if context_key else 'Context key missing in data'
            
            result = {'faithfulness': random.random(), 'answer_relevancy': random.random(), 'context_relevancy': random.random()}
            qa['question'].append(entry['question'])
            qa['ground_truth'].append(entry.get('ground_truth', 'No ground truth provided'))
            qa['contexts'].append(specific_context)
            qa['answer'].append(entry['answer'])

            for key in result:
                qa[key].append(result[key])

            pbar.update(1)

    with open(save_file, 'wb') as f:
        pickle.dump(qa, f)

def load_and_display_results():
    data_directory = 'data'
    data_file = "qa_data.pickle"
    save_file = os.path.join(data_directory, data_file)

    if os.path.exists(save_file):
        with open(save_file, 'rb') as f:
            data = pickle.load(f)
        df = pd.DataFrame(data)
        print(df)
    else:
        print(f"File {save_file} not found.")

if __name__ == "__main__":
    context_articles, test_set = load_data(sample_fraction=1)  # Ajuste conforme necessário para velocidade
    if context_articles and test_set:
        process_and_evaluate(context_articles, test_set)
        load_and_display_results()


100%|██████████| 50/50 [00:00<00:00, 114975.44it/s]

                                             question  \
0           What is Zeus know for in Greek mythology?   
1   How long had the First World War been over whe...   
2   How old was Messe when the First World War sta...   
3   How long had Angela Scoular been acting profes...   
4   What is the capacity of the stadium where Brun...   
5   Which stadium where Brunt played can hold more...   
6           In which country was Wilhelm Müller born?   
7   Which battle Wilhelm Müller fought in while in...   
8   How much time had passed between Wilhelm's ret...   
9   Which of Wilhelm's direct male descendants liv...   
10  In which Italian region did Pesce studied medi...   
11    Which country hosted the 1948 Leprosy Congress?   
12  When was the sports organization that the Turk...   
13  What was the founding date of the championship...   
14  How many years did the player who  top-scored ...   
15                    How many teams play in the NHL?   
16       Which hockey team won 

In [11]:
import os
import json
import pickle
import random
from tqdm import tqdm
import pandas as pd

def load_data(sample_fraction=1.0):  # Default para 1.0, que significa ler todos os dados
    data_directory = 'data'
    articles_file = os.path.join(data_directory, "context_articles.json")
    questions_file = os.path.join(data_directory, "test_questions.json")

    if os.path.exists(articles_file) and os.path.exists(questions_file):
        with open(articles_file, 'r') as f:
            context_articles = json.load(f)
        with open(questions_file, 'r') as f:
            test_set = json.load(f)

        # # Amostra uma fração dos dados para testes rápidos se necessário
        # if sample_fraction < 1.0:
        #     sample_size = max(1, int(len(test_set) * sample_fraction))
        #     test_set = random.sample(test_set, sample_size)

        return context_articles, test_set
    else:
        print("Files not found, check your dataset path.")
        return None, None

def process_and_evaluate(context_articles, test_set):
    data_directory = 'data'
    data_file = "qa_data.pickle"
    save_file = os.path.join(data_directory, data_file)

    if not os.path.exists(data_directory):
        os.makedirs(data_directory)

    qa = {'question': [], 'ground_truth': [], 'contexts': [], 'answer': [],
          'faithfulness': [], 'answer_relevancy': [], 'context_relevancy': []}

    with tqdm(total=len(test_set)) as pbar:
        for entry in test_set:
            result = {'faithfulness': random.random(), 'answer_relevancy': random.random(), 'context_relevancy': random.random()}
            qa['question'].append(entry['question'])
            qa['ground_truth'].append(entry.get('ground_truth', 'No ground truth provided'))
            qa['contexts'].append(context_articles)
            qa['answer'].append(entry['answer'])

            for key in result:
                qa[key].append(result[key])

            pbar.update(1)

    with open(save_file, 'wb') as f:
        pickle.dump(qa, f)

def load_and_display_results():
    data_directory = 'data'
    data_file = "qa_data.pickle"
    save_file = os.path.join(data_directory, data_file)

    if os.path.exists(save_file):
        with open(save_file, 'rb') as f:
            data = pickle.load(f)
        df = pd.DataFrame(data)
        print(df)
    else:
        print(f"File {save_file} not found.")

if __name__ == "__main__":
    context_articles, test_set = load_data(sample_fraction=0.1)  # Ajuste conforme necessário para velocidade
    if context_articles and test_set:
        process_and_evaluate(context_articles, test_set)
        load_and_display_results()


100%|██████████| 50/50 [00:00<00:00, 62751.41it/s]


                                             question  \
0           What is Zeus know for in Greek mythology?   
1   How long had the First World War been over whe...   
2   How old was Messe when the First World War sta...   
3   How long had Angela Scoular been acting profes...   
4   What is the capacity of the stadium where Brun...   
5   Which stadium where Brunt played can hold more...   
6           In which country was Wilhelm Müller born?   
7   Which battle Wilhelm Müller fought in while in...   
8   How much time had passed between Wilhelm's ret...   
9   Which of Wilhelm's direct male descendants liv...   
10  In which Italian region did Pesce studied medi...   
11    Which country hosted the 1948 Leprosy Congress?   
12  When was the sports organization that the Turk...   
13  What was the founding date of the championship...   
14  How many years did the player who  top-scored ...   
15                    How many teams play in the NHL?   
16       Which hockey team won 

In [8]:

def load_data(sample_fraction=0.01):
    data_directory = 'data'
    articles_file = os.path.join(data_directory, "context_articles.json")
    questions_file = os.path.join(data_directory, "test_questions.json")

    if os.path.exists(articles_file) and os.path.exists(questions_file):
        with open(articles_file, 'r') as f:
            context_articles = json.load(f)
        with open(questions_file, 'r') as f:
            test_set = json.load(f)
        
        # Amostra uma fração dos dados para testes rápidos
        if sample_fraction < 10.0:
            sample_size = max(1, int(len(test_set) * sample_fraction))
            test_set = random.sample(test_set, sample_size)

        return context_articles, test_set
    else:
        print("Files not found, check your dataset path.")
        return None, None

def process_and_evaluate(context_articles, test_set):
    data_directory = 'data'
    data_file = "qa_data.pickle"
    save_file = os.path.join(data_directory, data_file)

    if not os.path.exists(data_directory):
        os.makedirs(data_directory)

    qa = {'question': [], 'ground_truth': [], 'contexts': [], 'answer': [],
          'faithfulness': [], 'answer_relevancy': [], 'context_relevancy': []}

    with tqdm(total=len(test_set)) as pbar:
        for entry in test_set:
            result = {'faithfulness': random.random(), 'answer_relevancy': random.random(), 'context_relevancy': random.random()}
            qa['question'].append(entry['question'])
            qa['ground_truth'].append(entry.get('ground_truth', 'No ground truth provided'))
            qa['contexts'].append(context_articles)  # This might need optimization if too large
            qa['answer'].append(entry['answer'])

            for key in result:
                qa[key].append(result[key])

            pbar.update(1)

    with open(save_file, 'wb') as f:
        pickle.dump(qa, f)

def load_and_display_results():
    data_directory = 'data'
    data_file = "qa_data.pickle"
    save_file = os.path.join(data_directory, data_file)

    if os.path.exists(save_file):
        with open(save_file, 'rb') as f:
            data = pickle.load(f)
        df = pd.DataFrame(data)
        print(df)
    else:
        print(f"File {save_file} not found.")

if __name__ == "__main__":
    context_articles, test_set = load_data(sample_fraction=0.2)  # Adjust fraction as needed for speed
    if context_articles and test_set:
        process_and_evaluate(context_articles, test_set)
        load_and_display_results()


100%|██████████| 10/10 [00:00<00:00, 192399.27it/s]


                                            question  \
0  Who owned the Rochester Rhinos when Glenn left...   
1  How old was Messe when the First World War sta...   
2  Which of the battles Sherman fought during a w...   
3  Had the Chicago White Sox been around longer t...   
4  How many years were there between Pompeii's de...   
5  When was the company for which Tookey called a...   
6  Which stadium where Brunt played can hold more...   
7  How long had the First World War been over whe...   
8       What state did Glenn finish his career in?\n   
9  How long had Eddie Waitkus been playing profes...   

               ground_truth  \
0  No ground truth provided   
1  No ground truth provided   
2  No ground truth provided   
3  No ground truth provided   
4  No ground truth provided   
5  No ground truth provided   
6  No ground truth provided   
7  No ground truth provided   
8  No ground truth provided   
9  No ground truth provided   

                                         

In [44]:
import os
import json
import pickle
import random
import time
from tqdm import tqdm
import pandas as pd

def load_data():
    data_directory = 'data'
    articles_file = os.path.join(data_directory, "context_articles.json")
    questions_file = os.path.join(data_directory, "test_questions.json")

    # Carregar artigos de contexto e conjunto de teste
    if os.path.exists(articles_file) and os.path.exists(questions_file):
        with open(articles_file, 'r') as f:
            context_articles = json.load(f)
        with open(questions_file, 'r') as f:
            test_set = json.load(f)
        return context_articles, test_set
    else:
        print("Files not found, check your dataset path.")
        return None, None

def process_and_evaluate(context_articles, test_set):
    data_directory = 'data'
    data_file = "qa_data.pickle"
    save_file = os.path.join(data_directory, data_file)

    # Garantindo que o diretório de dados exista
    os.makedirs(data_directory, exist_ok=True)

    # Carregando ou inicializando o dicionário de QA
    if os.path.exists(save_file):
        with open(save_file, 'rb') as f:
            qa = pickle.load(f)
    else:
        qa = {
            'question': [],
            'ground_truth': [],
            'contexts': [],
            'answer': [],
            'faithfulness': [],
            'answer_relevancy': [],
            'context_relevancy': []
        }

    # Assumindo que 'test_set' é uma lista de dicionários
    with tqdm(total=len(test_set)) as pbar:
        for entry in test_set:
            if entry['question'] not in qa['question']:
                try:
                    # Simulação de avaliação
                    result = {'faithfulness': random.random(), 'answer_relevancy': random.random(), 'context_relevancy': random.random()}
                    
                    qa['question'].append(entry['question'])
                    qa['ground_truth'].append(entry['ground_truth'])
                    qa['contexts'].append(context_articles)  # Exemplo simplificado
                    qa['answer'].append(entry['answer'])
                    for key in result:
                        qa[key].append(result[key])

                    if len(qa['question']) % 5 == 0:
                        with open(save_file, 'wb') as f:
                            pickle.dump(qa, f)
                except Exception as e:
                    print(f"Error at index {len(qa['question'])}: {e}")
                    time.sleep(120)
                pbar.update(1)

        # Salvando os dados finais
        with open(save_file, 'wb') as f:
            pickle.dump(qa, f)

def load_and_display_results():
    data_directory = 'data'
    data_file = "qa_data.pickle"
    save_file = os.path.join(data_directory, data_file)

    if os.path.exists(save_file):
        with open(save_file, 'rb') as f:
            data = pickle.load(f)
        df = pd.DataFrame(data)
        print(df)
    else:
        print(f"Arquivo {save_file} não encontrado.")

if __name__ == "__main__":
    context_articles, test_set = load_data()
    if context_articles and test_set:
        process_and_evaluate(context_articles, test_set)
        load_and_display_results()


  0%|          | 0/50 [00:00<?, ?it/s]

Error at index 2: 'ground_truth'


  2%|▏         | 1/50 [02:00<1:38:00, 120.00s/it]

Error at index 3: 'ground_truth'


  4%|▍         | 2/50 [04:00<1:36:00, 120.01s/it]

...

Error at index 50: 'ground_truth'


 98%|█████████▊| 49/50 [1:38:00<02:00, 120.01s/it]

Error at index 51: 'ground_truth'


100%|██████████| 50/50 [1:40:00<00:00, 120.01s/it]


ValueError: All arrays must be of the same length

In [15]:
def setup_dataset():
    DATA_DIR = 'data'
    if not os.path.exists(DATA_DIR):
        os.makedirs(DATA_DIR)

    context_articles = "https://iirc-dataset.s3.us-west-2.amazonaws.com/context_articles.tar.gz"
    filename = 'context_articles.tar.gz'
    filepath = os.path.join(DATA_DIR, filename)

    if not os.path.exists(filepath):
        print(f"Downloading {filename}...")
        response = requests.get(context_articles)
        with open(filepath, 'wb') as f:
            f.write(response.content)
        print(f"{filename} downloaded.")
        
        with tarfile.open(filepath, 'r:gz') as tar:
            tar.extractall(path=DATA_DIR)
        print(f"{filename} extracted.")


In [37]:
import os
import pickle
import random
import time
from tqdm import tqdm

def process_and_evaluate():
    # Definindo o diretório e o arquivo de dados
    data_directory = 'data'
    data_file = "qa_data.pickle"
    save_file = os.path.join(data_directory, data_file)

    # Garantindo que o diretório de dados exista
    os.makedirs(data_directory, exist_ok=True)

    # Carregando ou inicializando o dicionário de QA
    if os.path.exists(save_file):
        with open(save_file, 'rb') as f:
            qa = pickle.load(f)
    else:
        qa = {
            'question': [],
            'ground_truth': [],
            'contexts': [],
            'answer': [],
            'faithfulness': [],
            'answer_relevancy': [],
            'context_relevancy': []
        }

    # Simulando dados de entrada para processamento
    dataset = [
        {'question': "What is AI?", 'ground_truth': "Study of intelligent agents.", 'contexts': "AI is used in various fields.", 'answer': "AI stands for Artificial Intelligence."}
    ]

    with tqdm(total=len(dataset)) as pbar:
        for entry in dataset:
            if entry['question'] not in qa['question']:
                try:
                    # Simulação de avaliação
                    result = {'faithfulness': random.random(), 'answer_relevancy': random.random(), 'context_relevancy': random.random()}
                    
                    qa['question'].append(entry['question'])
                    qa['ground_truth'].append(entry['ground_truth'])
                    qa['contexts'].append(entry['contexts'])
                    qa['answer'].append(entry['answer'])
                    for key in result:
                        qa[key].append(result[key])

                    if len(qa['question']) % 5 == 0:
                        with open(save_file, 'wb') as f:
                            pickle.dump(qa, f)
                except Exception as e:
                    print(f"Error at index {len(qa['question'])}: {e}")
                    time.sleep(120)
                pbar.update(1)

        with open(save_file, 'wb') as f:
            pickle.dump(qa, f)


In [38]:
def load_and_display_results():
    data_directory = 'data'
    data_file = "qa_data.pickle"
    save_file = os.path.join(data_directory, data_file)

    if os.path.exists(save_file):
        with open(save_file, 'rb') as f:
            data = pickle.load(f)
        df = pd.DataFrame(data)
        print(df)
    else:
        print(f"Arquivo {save_file} não encontrado.")

In [39]:
if __name__ == "__main__":
    setup_dataset()
    process_and_evaluate()
    # Carregar e exibir os resultados
    with open('qa_data.pickle', 'rb') as f:
        data = pickle.load(f)
    df = pd.DataFrame(data)
    print(df.describe())
    df.hist(figsize=(10,8), layout=(1,3))


  0%|          | 0/1 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'qa_data.pickle'